<a href="https://colab.research.google.com/github/ChemsSun/PFE/blob/main/Ho_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.api.types import is_numeric_dtype
import warnings
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree  import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import BernoulliNB
from lightgbm import LGBMClassifier
from sklearn.feature_selection import RFE
import itertools
from xgboost import XGBClassifier
from tabulate import tabulate
import random
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [33]:
train=pd.read_csv('/content/KDDTrain+.csv')
test=pd.read_csv('/content/KDDTest+.csv')
# train=train.sample(n=100,random_state=42)

In [34]:
def le(df):
    for col in df.columns:
        if df[col].dtype == 'object':
                label_encoder = LabelEncoder()
                df[col] = label_encoder.fit_transform(df[col])

le(train)
le(test)

In [35]:
from sklearn.utils import shuffle
# train = shuffle(train, random_state=42)
# test = shuffle(test, random_state=42)
x_train = train.drop(['class'], axis=1)
y_train = train['class']
x_test = test.drop(['class'], axis=1)
y_test = test['class']


In [36]:
x_train = x_train[['flag', 'src_bytes', 'dst_bytes', 'same_srv_rate', 'dst_host_same_srv_rate']]
x_test = x_test[['flag', 'src_bytes', 'dst_bytes', 'same_srv_rate', 'dst_host_same_srv_rate']]

In [37]:
scale=StandardScaler()
x_train=scale.fit_transform(x_train)
x_test=scale.fit_transform(x_test)

In [ ]:
# X_train = train.drop(['class'], axis=1)
# Y_train = train['class']
# X_train = X_train[['flag', 'src_bytes', 'dst_bytes', 'same_srv_rate', 'dst_host_same_srv_rate']]
# scale = StandardScaler()
# X_train = scale.fit_transform(X_train)
# x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, train_size=0.70,random_state=42)

In [57]:
# x_train_df = pd.DataFrame(x_train)
x_test_df = pd.DataFrame(x_test)
# y_train_df = pd.DataFrame(y_train)
y_test_df = pd.DataFrame(y_test)

# x_train_df.to_csv('x_train.csv', index=False)
x_test_df.to_csv('x_test.csv', index=False)
# y_train_df.to_csv('y_train.csv', index=False)
y_test_df.to_csv('y_test.csv', index=False)

In [38]:
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from collections import Counter
import math
import csv

def accuracy_percentage(y_true, y_pred):
    correct = sum(1 for true, pred in zip(y_true, y_pred) if true == pred)
    total = len(y_true)
    return (correct / total) * 100

def count_errors(y_true, y_pred):
    errors = 0
    for true, pred in zip(y_true, y_pred):
        if true != pred:
            errors += 1
    return errors

def calculate_accuracy(correct_predictions, total_predictions):
    return correct_predictions / total_predictions if total_predictions else 0

class OptunaKNN:
    def __init__(self, k):
        self.k = k
        self.distances = []

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X):
        predictions = [self._predict(x) for x in X]
        return predictions

    def _predict(self, x):
        distances = np.linalg.norm(self.X_train - x, axis=1)
        np.save('distances.npy', distances)
        k_indices = np.argsort(distances)[:self.k]
        np.save('k_indices.npy', k_indices)
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        np.save('k_nearest_labels.npy', k_nearest_labels)
        most_common = Counter(k_nearest_labels).most_common(1)[0][0]
        return most_common

    # def _predict(self, x):
    #     distances = [self.euclidean_distance(x, x_train_i) for x_train_i in self.X_train]
    #     self.distances.append(distances)
    #     np.save('distances.npy', distances)  # Sauvegarde les distances dans un fichier distances.npy
    #     k_indices = np.argsort(distances)[:self.k]
    #     np.save('k_indices.npy', k_indices)  # Sauvegarde les indices dans un fichier k_indices.npy
    #     k_nearest_labels = [self.y_train[i] for i in k_indices]
    #     np.save('k_nearest_labels.npy', k_nearest_labels)  # Sauvegarde les étiquettes dans un fichier k_nearest_labels.npy
    #     most_common = Counter(k_nearest_labels).most_common(1)[0][0]
    #     return most_common

    # def euclidean_distance(self, line1, line2):
    #     distance = 0
    #     for x in range(len(line1)):
    #         if ((type(line1[x]) == str) or (type(line2[x]) == str)):
    #             if (line1[x] == line2[x]):
    #                 distance = 0
    #             else:
    #                 distance += 1
    #         else:
    #             distance += pow((line1[x] - line2[x]), 2)
    #     return math.sqrt(distance)

In [40]:
import time
from sklearn.metrics import confusion_matrix, classification_report, f1_score

# Meilleur k trouvé
y_train = np.array(y_train, dtype=int)
y_test = np.array(y_test, dtype=int)

# Meilleur k trouvé
best_k = 1
best_clf = OptunaKNN(k=best_k)

# Entraînement du modèle
best_clf.fit(x_train, y_train)

# Prédictions sur les données de test
start_time = time.time()
predictions = best_clf.predict(x_test)
end_time = time.time()

# Ensure predictions are numeric array
predictions = np.array(predictions, dtype=int)

# Print types for debugging
print(f"Type of y_test: {y_test.dtype}")
print(f"Type of predictions: {predictions.dtype}")

# Calcul de la précision et des erreurs
best_accuracy = accuracy_percentage(y_test, predictions)
best_errors = count_errors(y_test, predictions)

# Calcul de la matrice de confusion
print("Accuracy:", best_accuracy)
print("Errors:", best_errors)
print("Time elapsed:", end_time - start_time)

# Print confusion matrix and classification report
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

Type of y_test: int64
Type of predictions: int64
Accuracy: 74.47214336408801
Errors: 5755
Time elapsed: 309.10406041145325
[[8491 4342]
 [1413 8298]]
              precision    recall  f1-score   support

           0       0.86      0.66      0.75     12833
           1       0.66      0.85      0.74      9711

    accuracy                           0.74     22544
   macro avg       0.76      0.76      0.74     22544
weighted avg       0.77      0.74      0.75     22544



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

# Calcul de la matrice de confusion et du rapport de classification
conf_matrix = confusion_matrix(y_test, predictions)
class_report = classification_report(y_test, predictions, target_names=target_names, output_dict=True)

# Tracé de la matrice de confusion avec une palette de couleurs personnalisée
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=target_names, yticklabels=target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Affichage du rapport de classification
class_report_df = pd.DataFrame(class_report).transpose()
print(class_report_df)


In [41]:
def adding(clf, x_train, y_train, instance, verification):
        # Incorrect predictio
        # Add the incorrectly predicted instance to the training data
    x_train = np.append(x_train, [instance], axis=0)  # Add instance to features
    y_train = np.append(y_train, [verification], axis=0)     # Add correct label to target
    clf.fit(x_train, y_train)  # Refit the classifier with the updated training set

    return clf, x_train, y_train

def remplacement(clf, x_train, y_train, x_test, y_test,nearest_neighbor_idx):

        # Replace the nearest neighbor in the training set with the test instance
        x_train[nearest_neighbor_idx] = x_test
        y_train[nearest_neighbor_idx] = y_test
        # Refit the classifier with the updated training set


        clf.fit(x_train, y_train)

        return clf, x_train, y_train

In [42]:
def count_predictions(clf,x_test,y_test):
  correct=0
  errors=0

  for instance, verification in zip(x_test, y_test):
    prediction = clf.predict([instance])[0]  # Predict label for the instance

    # Compare prediction with actual label
    if prediction == verification:
      correct += 1
    else:
      errors += 1
  return correct,errors

def find_nearest_neighbor(clf, x_train, instance):
    distances = np.linalg.norm(x_train - instance, axis=1)
    nearest_neighbor_idx = np.argmin(distances)
    distance = distances[nearest_neighbor_idx]
    return nearest_neighbor_idx, distance


def calculate_accuracy(correct_predictions, total_predictions):
    return correct_predictions / total_predictions

def ho_knn1(clf, x_train, y_train, x_test, y_test, threshold,counts):
    prediction = clf.predict([x_test])[0]
    if prediction == y_test:
        return clf, x_train, y_train, "correct"
    else:
        nearest_neighbor_idx, distance = find_nearest_neighbor(clf, x_train, x_test)
        if distance < threshold:
            clf, x_train, y_train= remplacement(clf, x_train, y_train, x_test, y_test, nearest_neighbor_idx)
            counts['replaced'] += 1
            return clf, x_train, y_train, "replaced"
        else:
            clf, x_train, y_train= adding(clf, x_train, y_train, x_test, y_test)
            counts['added'] += 1
            return clf, x_train, y_train,"added"



def ho_knn2(clf, x_train, y_train, x_test, y_test, threshold,counts):
    prediction = clf.predict([x_test])[0]
    if prediction == y_test:
        return clf, x_train, y_train, "correct"
    else:
        nearest_neighbor_idx, distance = find_nearest_neighbor(clf, x_train, x_test)
        if distance > threshold:
            clf, x_train, y_train= remplacement(clf, x_train, y_train, x_test, y_test, nearest_neighbor_idx)
            counts['replaced'] += 1
            return clf, x_train, y_train, "replaced"
        else:
            clf, x_train, y_train= adding(clf, x_train, y_train, x_test, y_test)
            counts['added'] += 1
            return clf, x_train, y_train,"added"

In [30]:
x_train = train.drop(['class'], axis=1)
y_train = train['class']
x_test = test.drop(['class'], axis=1)
y_test = test['class']
scale=StandardScaler()
x_train=scale.fit_transform(x_train)
x_test=scale.fit_transform(x_test)

In [43]:
import random
import time
best_k=1
threshold =0.0445498784733719
# threshold=random.uniform(0.01227,3.23779)
best_clf1 = OptunaKNN(k=best_k)

best_clf1.fit(x_train, y_train)

counts = {'replaced': 0, 'added': 0}
correct_predictions = 0
total_predictions = 0
correct_counts = []
replaced_counts = []
accuracies_code2 = []
added_counts = []
instances_processed = []

start_time = time.time()
#Strategy 1
for instance, actual_label in zip(x_test, y_test):
    best_clf1, x_train, y_train, status = ho_knn1(best_clf1, x_train, y_train, instance, actual_label, threshold,counts)
    if status == "correct":
        correct_predictions += 1
    total_predictions += 1
    # print(f"Instance processed, status: {status}")
end_time = time.time()

# Strategy 2
# for instance, actual_label in zip(x_test, y_test):
#     best_clf1, x_train, y_train, status = ho_knn2(best_clf1, x_train, y_train, instance, actual_label, threshold,counts)
#     if status == "correct":
#         correct_predictions += 1
#     total_predictions += 1
# end_time = time.time()


print(f"Total replacements: {counts['replaced']}")
print(f"Total additions: {counts['added']}")
accuracy_code2 = calculate_accuracy(correct_predictions, total_predictions)
accuracies_code2.append(accuracy_code2)

print(f"Accuracy: {accuracy_code2 * 100:}%")
print(f"Time taken: {end_time - start_time:.2f} s")
print('ur threshhold is: ',threshold)


Total replacements: 3515
Total additions: 372
Accuracy: 82.7581618168914%
Time taken: 318.04 s
ur threshhold is:  0.0445498784733719


In [44]:
import pickle

In [45]:
pickle.dump(best_clf1, open('hoknn_model.pkl', 'wb'))

In [46]:
model_load=pickle.load(open('hoknn_model.pkl', 'rb'))

In [61]:
x_testt=pd.read_csv('/content/x_test.csv')
y_testt=pd.read_csv('/content/y_test.csv')

In [62]:
y_testt

,0
0,0
1,0
2,1
3,0
4,0
...,...
22539,1
22540,1
22541,0
22542,1


In [49]:
input_data = (0.66225582,0.0054731,-0.09689597,0.62948754,0.00293425)

# changing the input_data to numpy array
input_data_as_numpy_array = np.asarray(input_data)

# reshape the array as we are predicting for one instance
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

prediction = model_load.predict(input_data_reshaped)
print(prediction)

if (prediction[0] == 0):
  print('attaque')
else:
  print('normal')

[1]
normal


In [50]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.5 MB/s eta 0:00:00


In [64]:
!wget -q -O - ipv4.icanhazip.com

35.199.149.145


In [65]:
! streamlit run IDS.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.199.149.145:8501

npx: installed 22 in 3.36s
your url is: https://hot-badgers-shave.loca.lt
2024-05-31 20:52:57.656 Uncaught app exception
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runtime/scriptrunner/script_runner.py", line 600, in _run_script
    exec(code, module.__dict__)
  File "/content/IDS.py", line 7, in <module>
    df=pd.read_csv('/content/x_test.csv')
NameError: name 'pd' is not defined
[1]
[1]
[1]
[0]
2024-05-31 21:00:53.262 Uncaught app exception
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runtime/scriptrunner/script_runner.py", line 600, in _run_script
    exec(code, module.__dict__)
  File "/content/IDS.py", line 14, in <module>
    st.write(df_combined)
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runt